In [ ]:
# 야생동물 데이터 다운로드
# https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=645

# COCO 데이터셋
# https://soonminhwang.github.io/rgbt-ped-detection/

In [1]:
import json
import shutil
import os 
import yaml
import cv2
import zipfile
import time
from glob import glob
from tqdm import tqdm
from openpyxl import Workbook
from scipy.sparse import data
from sklearn.model_selection import train_test_split

In [2]:

# 종혁 : 코랩 실행시
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 이미지 리사이징 처리한 야생동물 데이터불러오기

대상 야생동물 : 고라니, 멧돼지, 너구리, 반달가슴곰


In [3]:
# 압축 파일 경로를 설정해주세요.
# 리스트 형태이며, 여러 파일도 가능합니다.
zip_file_path_image =  ['/content/drive/MyDrive/[SeSAC] Final_PJT 공유폴더/03. 파이널_야생동물 객체 탐지, 추적 모델/데이터셋/1.Training/원천데이터/TS_01.고라니_리사이징완료.zip',
                       '/content/drive/MyDrive/[SeSAC] Final_PJT 공유폴더/03. 파이널_야생동물 객체 탐지, 추적 모델/데이터셋/1.Training/원천데이터/TS_02.멧돼지_리사이징완료.zip',
                       '/content/drive/MyDrive/[SeSAC] Final_PJT 공유폴더/03. 파이널_야생동물 객체 탐지, 추적 모델/데이터셋/1.Training/원천데이터/TS_03.너구리_리사이징완료.zip',
                       '/content/drive/MyDrive/[SeSAC] Final_PJT 공유폴더/03. 파이널_야생동물 객체 탐지, 추적 모델/데이터셋/1.Training/원천데이터/TS_06.반달가슴곰_리사이징완료.zip']

zip_file_path_json = ['/content/drive/MyDrive/[SeSAC] Final_PJT 공유폴더/03. 파이널_야생동물 객체 탐지, 추적 모델/데이터셋/1.Training/라벨링데이터/TL_01.고라니.zip',
                      '/content/drive/MyDrive/[SeSAC] Final_PJT 공유폴더/03. 파이널_야생동물 객체 탐지, 추적 모델/데이터셋/1.Training/라벨링데이터/TL_02.멧돼지.zip',
                      '/content/drive/MyDrive/[SeSAC] Final_PJT 공유폴더/03. 파이널_야생동물 객체 탐지, 추적 모델/데이터셋/1.Training/라벨링데이터/TL_03.너구리.zip',
                      '/content/drive/MyDrive/[SeSAC] Final_PJT 공유폴더/03. 파이널_야생동물 객체 탐지, 추적 모델/데이터셋/1.Training/라벨링데이터/TL_06.반달가슴곰.zip']
                      

# 본 Main 코드 폴더가 있는 폴더 경로를 지정해주세요.
# 압축이 풀린"이미지" 파일은 Main 코드가 있는 폴더 -> Dataset -> images 폴더로 들어갑니다.
# 압축이 풀린"json" 파일은 Main 코드가 있는 폴더 -> Dataset -> json 폴더로 들어갑니다.
!mkdir /content/train
!mkdir /content/train/Dataset
!mkdir /content/train/Dataset/images
!mkdir /content/train/Dataset/json_day_night
!mkdir /content/train/Dataset/labels

main_path = '/content/train/'
google_drive_path = '/content/drive/MyDrive/[SeSAC] Final_PJT 공유폴더/03. 파이널_야생동물 객체 탐지, 추적 모델/코드/03. yolov7_train/'

In [4]:
# 고라니 테스트 이미지 압축 해제

for i in tqdm(range(len(zip_file_path_image))):
  zip_file_image = zipfile.ZipFile(zip_file_path_image[i])
  file_list = zip_file_image.namelist()
  for file_name in tqdm(file_list, desc=f'Extracting files ({i}/{2*len(zip_file_path_image)}'):
    zip_file_image.extract(file_name, main_path + 'Dataset/images')

  zip_file_json = zipfile.ZipFile(zip_file_path_json[i])
  file_list = zip_file_json.namelist()
  for file_name in tqdm(file_list, desc = f'Extracting files ({i}/{2*len(zip_file_path_image)}'):
    zip_file_json.extract(file_name, main_path + 'Dataset/json_day_night')


Extracting files (0/8: 100%|██████████| 31074/31074 [00:21<00:00, 1421.63it/s]

Extracting files (1/8: 100%|██████████| 30700/30700 [00:19<00:00, 1595.25it/s]

Extracting files (2/8: 100%|██████████| 13743/13743 [00:05<00:00, 2352.23it/s]

Extracting files (3/8: 100%|██████████| 14960/14960 [00:08<00:00, 1688.27it/s]

100%|██████████| 4/4 [01:26<00:00, 21.54s/it]


In [5]:
# 파일 개수 확인

import os

# Set folder path
source_folder = '/content/train/Dataset/images'

# Get list of files in folder
files = os.listdir(source_folder)

# Get number of files in folder
num_files = len(files)

print(f'The folder {source_folder} contains {num_files} files.')

The folder /content/train/Dataset/images contains 90477 files.


In [6]:
# 파일 개수 확인

import os

# Set folder path
source_folder = '/content/train/Dataset/json_day_night'

# Get list of files in folder
files = os.listdir(source_folder)

# Get number of files in folder
num_files = len(files)

print(f'The folder {source_folder} contains {num_files} files.')

The folder /content/train/Dataset/json_day_night contains 90477 files.


# 주간 / 야간 json 데이터 분리

In [7]:
import os
import json
import shutil

!mkdir /content/train/Dataset/night_json
!mkdir /content/train/Dataset/day_json

# 디렉토리 내의 모든 json 파일 목록을 가져오기
file_list = os.listdir('/content/train/Dataset/json_day_night')

for file_name in file_list:
  if file_name.endswith('.json'): # json 파일일 경우
    with open('/content/train/Dataset/json_day_night/' + file_name, 'r') as f:
      data = json.load(f)
      if data['images'][0]['day'] == 'night':
        shutil.copy('/content/train/Dataset/json_day_night/' + file_name, '/content/train/Dataset/night_json/'+file_name)
      elif data['images'][0]['day'] == 'day':
        shutil.copy('/content/train/Dataset/json_day_night/' + file_name, '/content/train/Dataset/day_json/'+file_name)


In [8]:
# 야간 파일 개수 확인

import os

# Set folder path
source_folder = '/content/train/Dataset/night_json'

# Get list of files in folder
files = os.listdir(source_folder)

# Get number of files in folder
num_files = len(files)

print(f'The folder {source_folder} contains {num_files} files.')

The folder /content/train/Dataset/night_json contains 42222 files.


In [9]:
# 주간 파일 개수 확인

import os

# Set folder path
source_folder = '/content/train/Dataset/day_json'

# Get list of files in folder
files = os.listdir(source_folder)

# Get number of files in folder
num_files = len(files)

print(f'The folder {source_folder} contains {num_files} files.')

The folder /content/train/Dataset/day_json contains 48255 files.


In [10]:
# 파일 개수 확인

import os

# Set folder path
source_folder = '/content/train/Dataset/labels'

# Get list of files in folder
files = os.listdir(source_folder)

# Get number of files in folder
num_files = len(files)

print(f'The folder {source_folder} contains {num_files} files.')

The folder /content/train/Dataset/labels contains 0 files.


# 야생동물 JSON 데이터 txt 파일로 수정

In [11]:
# json 형태의 annotation 데이터를 YOLOv5 학습 형태에 맞게 txt 로 변환한다.
# 변환된 데이터는 ./Dataset/labels 에 저장됩니다.

classification = []
count = []

TOP = 0
LEFT = 1
BOTTOM = 2
RIGHT = 3

SQURE = 1

class Label:

	def __init__(self, fileName, imagePath):
		data = self.parse_json_data(fileName)
		self.set_data(data, fileName, imagePath)

	def parse_json_data(self, fileName):
		with open(fileName, encoding='utf-8-sig') as jsonFile:
			jsonData = json.load(jsonFile)
		return jsonData

	def set_data(self, data, fileName, imagePath):
		self.fileName = '.'.join(os.path.basename(fileName).split('.')[0:-1])
		self.width = int(data['images'][0]['width'])
		self.height = int(data['images'][0]['height'])
		self.annotations = data['annotations']

	def get_bbox_point(self, bbox):
		top = 10000
		left = 10000
		bottom = -1
		right = -1

		for i in bbox:
			if (i[0] < left): left = i[0]
			if (i[0] > right): right = i[0]
			if (i[1] < top): top = i[1]
			if (i[1] > bottom): bottom = i[1]

		return ((top, left, bottom, right))

	def get_segment_point(self, seg):
		top = 10000
		left = 10000
		bottom = -1
		right = -1

		for i in range(len(seg)):
			if (i % 2 == 0):
				if (seg[i] < left): left = seg[i]
				if (seg[i] > right): right = seg[i]
			else:
				if (seg[i] < top): top = seg[i]
				if (seg[i] > bottom): bottom = seg[i]

		return ((top, left, bottom, right))

	def get_shape_point(self, shape):
		try: #종혁 : 추가
			if (shape['bbox'] and len(shape['bbox']) != 0):
				return self.get_bbox_point(shape['bbox'])
			elif (shape['Segmentation'] and len(shape['Segmentation']) != 0):
				return self.get_segment_point(shape['Segmentation'][0])
			else:
				return (0, 0, 0, 0)
		except: #종혁 : 추가
			return (0, 0, 0, 0) #종혁 : 추가

	def point_to_txt(self, point):
		if (point[0]) not in classification:
			classification.append(point[0])
			count.append(0)
		idx = classification.index(point[0])
		count[idx] += 1

		if (SQURE):
			r = (self.width - self.height) / 2
			w_center = ((point[1][LEFT] + point[1][RIGHT]) / 2) / self.width
			h_center = (((point[1][TOP] + point[1][BOTTOM]) / 2) + r) / self.width
			w_len = (point[1][RIGHT] - point[1][LEFT]) / self.width
			h_len = (point[1][BOTTOM] - point[1][TOP]) / self.width
		else:
			w_center = ((point[1][LEFT] + point[1][RIGHT]) / 2) / self.width
			h_center = ((point[1][TOP] + point[1][BOTTOM]) / 2) / self.height
			w_len = (point[1][RIGHT] - point[1][LEFT]) / self.width
			h_len = (point[1][BOTTOM] - point[1][TOP]) / self.height
		return (idx, w_center, h_center, w_len, h_len)

	def convert_data(self):
		self.points = []
		self.txt = []
		for shape in self.annotations:
			self.points.append((shape['species'], self.get_shape_point(shape)))
		for point in self.points:
			self.txt.append(self.point_to_txt(point))

	def write_data(self, path):
		with open(path + self.fileName + ".txt", 'w') as f:
			for line in self.txt:
				f.write(' '.join(map(str, line)) + "\n")

def convert_data(file, targetPath, imagePath):
	target = Label(file, imagePath)
	target.convert_data()
	target.write_data(targetPath)

json_count = [0]
def convert(targetPath, src):
    # 원코드
    # from tqdm import tqdm
    # data_list = os.listdir(targetPath)
    # print(data_list) #추가
    # for line in data_list:
    #     print(os.path.join(targetPath, line)) # 추가
    #     if os.path.isdir(os.path.join(targetPath, line) + '/'):
    #         convert(os.path.join(targetPath, line), src)
    #         # 종혁 : 재귀적 함수? 와.. 이렇게 짜는 거구나.
    #         # 근데 그래서 속도가 느릴 거 같은데..
    #     elif '.json' in line:
    #         json_count[0] += 1
    #         print(json_count[0], "Data Convert:", line, end='\r')
    #         # shutil.copyfile(targetPath + "/" + line, src + "/json/" + line)
    #         # 비효율적인 코드 삭제 ==> 대신 OriginData 폴더와 json 폴더에 TL 파일들 넣어야 함
    #         convert_data(targetPath + "/" + line, src + "/labels/", src + "/images/")
    #     else:
    #        pass
    
    from tqdm import tqdm
    
    path = src + '/night_json'
    data_list = os.listdir(path)
    json_count = [0]
    
    for line in tqdm(data_list):
        json_count[0] += 1
        convert_data(path + '/' + line, src + '/labels/', src + '/images/')
        
    

image_count = [0]
def copy_image(targetPath, src):
	data_list = os.listdir(targetPath)
	for line in data_list:
		if os.path.isdir(os.path.join(targetPath, line)):
			copy_image(os.path.join(targetPath, line), src)
		elif '.jpg' in line:
			image_count[0] += 1
			print(image_count[0], "Image Copy:", line, end='\r')
			shutil.copyfile(targetPath + "/" + line, src + "/images/" + line)
		else:
			pass

def write_yaml(target):
	data = {}
	data["names"] = classification
	data["nc"] = len(classification)
	data["train"] = "../Dataset/train.txt"
	data["val"] = "../Dataset/valid.txt"
	data["test"] = "../Dataset/test.txt"
	with open(target, 'w', encoding='utf-8-sig') as f:
		yaml.dump(data, f, allow_unicode=True)

def run_convert(targetPath, src, copy):
	if not os.path.isdir(src + "/images"):
		os.makedirs(src + "/images/")
	if not os.path.isdir(src + "/labels"):
		os.makedirs(src + "/labels")
	if not os.path.isdir(src + "/json"):
		os.makedirs(src + "/json")
	# if copy:
	# 	copy_image(targetPath, src)
	# 비효율적인 코드 삭제 ==> 대신 image 폴더에 직접 이미지 넣어야 함
	convert(targetPath, src)
	write_yaml(os.path.join(src, "data.yaml"))
	for i in range(len(classification)):
		print(classification[i], "is counted:", count[i])



# 로컬 실행시
# import time
# start = time.time()
# run_convert("./OriginData", "./Dataset", False)
# end = time.time()
# print(f'{end - start:.5f} sec')


# 코랩 실행시
## 종혁 : 경로 재설정, 소요시간 기록
## run_convert(,,False) : False = 복사를 하지 않음
main_path = main_path
start = time.time()

run_convert("OriginData", main_path + "Dataset", False)
# 메인 코드 : 컨버팅

end = time.time()
print(f'{end - start:.5f} sec')

100%|██████████| 42222/42222 [00:06<00:00, 6915.79it/s]

너구리 is counted: 5056
멧돼지 is counted: 17236
고라니 is counted: 14141
노루 is counted: 764
반달가슴곰 is counted: 7484
멧토끼 is counted: 2
6.13761 sec


In [13]:
# 파일 개수 확인

import os

# Set folder path

source_folder = '/content/train/Dataset/labels'

# Get list of files in folder
files = os.listdir(source_folder)

# Get number of files in folder
num_files = len(files)

print(f'The folder {source_folder} contains {num_files} files.')

The folder /content/train/Dataset/labels contains 42222 files.


# 야생동물 데이터 class id (숫자) 인덱스 값 바꾸기

In [14]:
# 불필요한 노루 객체의 txt 데이터 삭제하기

import os

# 특정 폴더 내의 모든 txt 파일 가져오기
folder_path = '/content/train/Dataset/labels'
txt_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]

# 파일별로 <class-id> 확인하고, 4이면 삭제 수행
num_deleted_files = 0
for txt_file in txt_files:
    with open(os.path.join(folder_path, txt_file), 'r') as f:
        lines = f.readlines()
    class_ids = [int(line.strip().split()[0]) for line in lines]
    if 3 in class_ids :
        os.remove(os.path.join(folder_path, txt_file))
        num_deleted_files += 1

print(f"{num_deleted_files}개의 파일이 삭제되었습니다.")

722개의 파일이 삭제되었습니다.


In [15]:
# 불필요한 노루 객체의 txt 데이터 삭제하기

import os

# 특정 폴더 내의 모든 txt 파일 가져오기
folder_path = '/content/train/Dataset/labels'
txt_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]

# 파일별로 <class-id> 확인하고, 4이면 삭제 수행
num_deleted_files = 0
for txt_file in txt_files:
    with open(os.path.join(folder_path, txt_file), 'r') as f:
        lines = f.readlines()
    class_ids = [int(line.strip().split()[0]) for line in lines]
    if 5 in class_ids :
        os.remove(os.path.join(folder_path, txt_file))
        num_deleted_files += 1

print(f"{num_deleted_files}개의 파일이 삭제되었습니다.")

2개의 파일이 삭제되었습니다.


In [16]:
import os

# number conversion function
def convert_class_id(class_id):
    if class_id == 0 :
        return 3
    elif class_id == 1 :
        return 1    
    elif class_id == 2 :
        return 0    
    elif class_id == 4 :
        return 2    
    # else:
    #     return 5


# Get all txt files in a specific folder
folder_path = '/content/train/Dataset/labels'
txt_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]

# Counter for files that caused errors
num_error_files = 0

# Counter for files that were successfully modified
num_success_files = 0

# Perform <class-id> conversion per file
for txt_file in txt_files:
    with open(os.path.join(folder_path, txt_file), 'r') as f:
        lines = f.readlines()
    new_lines = []
    try:
        for line in lines:
            class_id, x, y, width, height = map(float, line.strip().split(' '))
            new_class_id = convert_class_id(int(class_id))
            new_lines.append(f'{new_class_id} {x} {y} {width} {height}\n')
        with open(os.path.join(folder_path, txt_file), 'w') as f:
            f.writelines(new_lines)
            num_success_files += 1
    except ValueError:
        # Increment counter for files that caused errors
        num_error_files += 1
        continue

print(f'Total number of files that caused errors: {num_error_files}')
print(f'Total number of files successfully modified: {num_success_files}')


Total number of files that caused errors: 0
Total number of files successfully modified: 41498


In [17]:
# 특정 클래스 id 를 가진 텍스트 파일 조회하기

folder_path = '/content/train/Dataset/labels'  # 조회할 폴더 경로

num_files = 0  # 조회된 파일 개수
found_files = []  # 클래스 아이디가 발견된 파일 목록

for file_name in os.listdir(folder_path):
    if file_name.endswith('.txt'):  # 확장자가 .txt인 파일들만 조회
        with open(os.path.join(folder_path, file_name), 'r') as f:
            lines = f.readlines()
            found_class_id = False
            for line in lines:
                if len(line.split()) >= 1:
                    class_id = int(line.split()[0])
                    if class_id == 1:  # 클래스 아이디가 4인 파일만 개수 증가
                        num_files += 1
                        found_class_id = True
                        break  # 이 파일은 더 이상 조회할 필요 없음
            if found_class_id:
                found_files.append(file_name)
            # else:
            #     print(f'Warning: No object with class-id 1 found in file {file_name}')

print(f'Total number of files with <class-id>4</class-id>: {num_files}')
print(f'Files with <class-id>4</class-id>: {found_files}')


Total number of files with <class-id>4</class-id>: 16567
Files with <class-id>4</class-id>: ['A02_F02_C081_C_211103_4027_20S_000009.412.txt', 'A02_H18_C052_C_210910_0280_20S_000010.877.txt', 'A02_H20_C014_C_211110_1005_20S_000003.879.txt', 'A02_H20_C014_C_211110_1120_20S_000009.138.txt', 'A02_H20_C012_C_211110_4228_20S_000014.112.txt', 'A02_F02_C081_C_211104_4010_20S_000019.240.txt', 'A02_H20_C014_C_211110_1131_20S_000000.971.txt', 'A02_H20_C012_C_211115_5009_20S_000019.135.txt', 'A02_H20_C014_C_211110_1003_20S_000011.224.txt', 'A02_F02_C081_C_211103_4011_20S_000016.302.txt', 'A02_H20_C023_C_211102_6087_20S_000012.912.txt', 'A02_H20_C014_C_211110_1190_20S_000000.142.txt', 'A02_H20_C014_C_211110_1168_20S_000000.749.txt', 'A02_H29_C004_C_210927_0001_20S_000007.718.txt', 'A02_H20_C013_C_211110_3245_20S_000009.407.txt', 'A02_F02_C081_C_211104_4010_20S_000003.842.txt', 'A02_H20_C014_C_211110_1218_20S_000007.438.txt', 'A02_H20_C013_C_211005_0077_20S_000009.686.txt', 'A02_H20_C014_C_211110_10

# KAIST dataset 세팅하기

In [18]:
# 압축 파일 경로를 설정해주세요.
# 리스트 형태이며, 여러 파일도 가능합니다.

zip_file_path_kaistimages = ['/content/drive/MyDrive/[SeSAC] Final_PJT 공유폴더/03. 파이널_야생동물 객체 탐지, 추적 모델/데이터셋/KAIST_resize_images.zip']


zip_file_path_kaistlabels = ['/content/drive/MyDrive/[SeSAC] Final_PJT 공유폴더/03. 파이널_야생동물 객체 탐지, 추적 모델/데이터셋/KAIST_remake_labels.zip']


# 본 Main 코드 폴더가 있는 폴더 경로를 지정해주세요.
# 압축이 풀린"이미지" 파일은 Main 코드가 있는 폴더 -> Dataset -> cocoimages 폴더로 들어갑니다.
# 압축이 풀린"json" 파일은 Main 코드가 있는 폴더 -> Dataset -> cocolabels 폴더로 들어갑니다.
!mkdir /content/train/kaistDataset
!mkdir /content/train/kaistDataset/kaistimages
!mkdir /content/train/kaistDataset/kaistlabels

main_path = '/content/train/'
google_drive_path = '/content/drive/MyDrive/[SeSAC] Final_PJT 공유폴더/03. 파이널_야생동물 객체 탐지, 추적 모델/코드/03. yolov7_train/'

In [19]:
# kaist data 압축 해제

for i in tqdm(range(len(zip_file_path_kaistimages))):
  zip_file_kaistimages = zipfile.ZipFile(zip_file_path_kaistimages[i])
  file_list = zip_file_kaistimages.namelist()
  for file_name in tqdm(file_list, desc=f'Extracting files ({i}/{2*len(zip_file_path_kaistimages)}'):
    zip_file_kaistimages.extract(file_name, main_path + 'kaistDataset/kaistimages')

  zip_file_kaistlabels = zipfile.ZipFile(zip_file_path_kaistlabels[i])
  file_list = zip_file_kaistlabels.namelist()
  for file_name in tqdm(file_list, desc = f'Extracting files ({i}/{2*len(zip_file_path_kaistimages)}'):
    zip_file_kaistlabels.extract(file_name, main_path + 'kaistDataset/kaistlabels')

Extracting files (0/2: 100%|██████████| 32748/32748 [00:12<00:00, 2628.20it/s]

100%|██████████| 1/1 [00:18<00:00, 18.73s/it]


In [ ]:
# 클래스 명 class_names = ["person", "people", "cyclist", "person?"]

# 객체가 조회된 파일 개수 15817 files.

# 객체별 파일 개수

## person : 12645
## people : 4536
## cyclist : 2028
## person? : 481

# 클래스별로 분리 하고 데이터 감축하기

In [20]:
!mkdir /content/train/DistDataset
!mkdir /content/train/DistDataset/first_class_labels

In [21]:
import os
import shutil

txt_folder = '/content/train/kaistDataset/kaistlabels/remake_labels'
class_folder = '/content/train/DistDataset/first_class_labels/'

for class_id_to_copy in range(5):
    num_files = 0
    os.makedirs(os.path.join(class_folder, str(class_id_to_copy)), exist_ok=True)
    for filename in os.listdir(txt_folder):
        if filename.endswith('.txt'):
            with open(os.path.join(txt_folder, filename), 'r') as f:
                lines = f.readlines()
                if not lines:  # Skip the file if the file is empty
                    continue
                class_id = int(lines[0].split()[0])
                if class_id != class_id_to_copy:
                    continue
            print(f'Copying file {filename} to class {class_id_to_copy} folder...')
            shutil.copy(os.path.join(txt_folder, filename), os.path.join(class_folder, str(class_id_to_copy), filename))
            num_files += 1
    print(f'Total {num_files} files were copied to class {class_id_to_copy} folder.')
    print("--------------------------------------------------")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Copying file set03-V000-I00039.txt to class 0 folder...
Copying file set03-V001-I01228.txt to class 0 folder...
Copying file set11-V001-I00368.txt to class 0 folder...
Copying file set05-V000-I00477.txt to class 0 folder...
Copying file set04-V000-I02468.txt to class 0 folder...
Copying file set05-V000-I00497.txt to class 0 folder...
Copying file set05-V000-I01269.txt to class 0 folder...
Copying file set10-V001-I01631.txt to class 0 folder...
Copying file set05-V000-I00942.txt to class 0 folder...
Copying file set05-V000-I00627.txt to class 0 folder...
Copying file set03-V001-I00770.txt to class 0 folder...
Copying file set11-V001-I00090.txt to class 0 folder...
Copying file set04-V000-I01844.txt to class 0 folder...
Copying file set03-V000-I03190.txt to class 0 folder...
Copying file set03-V001-I00177.txt to class 0 folder...
Copying file set03-V001-I00821.txt to class 0 folder...
Copying file set11-V000-I01747.txt to class 0 folder...
Copying file

In [22]:
import os

# specify the path to the folder
path = "/content/train/DistDataset/first_class_labels"

# loop through all folders and subfolders in the path
for foldername, subfolders, filenames in os.walk(path):

    # count the number of files in each folder
    num_files = len(filenames)

    # print the folder name and number of files
    print(f"{foldername}: {num_files}")

/content/train/DistDataset/first_class_labels: 0
/content/train/DistDataset/first_class_labels/4: 0
/content/train/DistDataset/first_class_labels/2: 1268
/content/train/DistDataset/first_class_labels/3: 265
/content/train/DistDataset/first_class_labels/1: 2728
/content/train/DistDataset/first_class_labels/0: 11556


In [23]:
shutil.rmtree('/content/train/DistDataset/first_class_labels/4')

In [31]:
# # "person?" 클래스 데이터 삭제
# shutil.rmtree('/content/train/DistDataset/first_class_labels/3')

In [24]:
# 폴더 파일들 한 폴더로 모으기
import os
import shutil

# specify the path to the source folder
src_path = "/content/train/DistDataset/first_class_labels"

# specify the path to the destination folder
dst_path = "/content/train/DistDataset/merged_files"

# create the destination folder if it doesn't already exist
if not os.path.exists(dst_path):
    os.makedirs(dst_path)

# loop through all folders and subfolders in the source path
for foldername, subfolders, filenames in os.walk(src_path):

    # loop through all files in the folder
    for filename in filenames:

        # get the full path of the file
        file_path = os.path.join(foldername, filename)

        # get the new destination path for the file
        new_file_path = os.path.join(dst_path, filename)

        # copy the file to the new destination
        shutil.copy(file_path, new_file_path)

In [25]:
# 클래스명 수정 
# 0, 1 = person => 4
# 2 = cyclist => 5

import os

# number conversion function
def convert_class_id(class_id):
    if class_id == 0:
        return 4
    elif class_id == 1:
        return 4
    else:
        return 5


# Get all txt files in a specific folder
folder_path = '/content/train/DistDataset/merged_files'
txt_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]

# Counter for files that caused errors
num_error_files = 0

# Counter for files that were successfully modified
num_success_files = 0

# Perform <class-id> conversion per file
for txt_file in txt_files:
    with open(os.path.join(folder_path, txt_file), 'r') as f:
        lines = f.readlines()
    new_lines = []
    try:
        for line in lines:
            class_id, x, y, width, height = map(float, line.strip().split(' '))
            new_class_id = convert_class_id(int(class_id))
            new_lines.append(f'{new_class_id} {x} {y} {width} {height}\n')
        with open(os.path.join(folder_path, txt_file), 'w') as f:
            f.writelines(new_lines)
            num_success_files += 1
    except ValueError:
        # Increment counter for files that caused errors
        num_error_files += 1
        continue

print(f'Total number of files that caused errors: {num_error_files}')
print(f'Total number of files successfully modified: {num_success_files}')


Total number of files that caused errors: 0
Total number of files successfully modified: 15817


In [28]:
folder_path = '/content/train/DistDataset/merged_files'

num_files = 0
found_files = []
error_files = []

for file_name in os.listdir(folder_path):
    if file_name.endswith('.txt'):
        try:
            with open(os.path.join(folder_path, file_name), 'r') as f:
                lines = f.readlines()
                found_class_id = False
                for line in lines:
                    if len(line.split()) >= 1:
                        try:
                            class_id = int(line.split()[0])
                        except ValueError:
                            continue # skip to next line if the first word is not a valid integer
                        if class_id == 4 :
                            num_files += 1
                            found_class_id = True
                            break
                if found_class_id:
                    found_files.append(file_name)
        except:
            error_files.append(file_name)

print(f'Total number of files with <class-id>4</class-id>: {num_files}')
print(f'Files with <class-id>4</class-id>: {found_files}')
print(f'Number of error files: {len(error_files)}')
print(f'Error files: {error_files}')


Total number of files with <class-id>4</class-id>: 14931
Files with <class-id>4</class-id>: ['set11-V001-I00453.txt', 'set09-V000-I02503.txt', 'set05-V000-I01881.txt', 'set09-V000-I02120.txt', 'set11-V000-I01946.txt', 'set03-V000-I01808.txt', 'set11-V000-I00426.txt', 'set10-V001-I04004.txt', 'set11-V000-I00808.txt', 'set10-V000-I01662.txt', 'set10-V000-I01285.txt', 'set04-V001-I01377.txt', 'set10-V001-I02817.txt', 'set04-V001-I01552.txt', 'set05-V000-I02379.txt', 'set03-V000-I03621.txt', 'set04-V000-I02239.txt', 'set05-V000-I01214.txt', 'set09-V000-I01375.txt', 'set10-V001-I01376.txt', 'set10-V000-I01553.txt', 'set03-V000-I01796.txt', 'set04-V000-I02792.txt', 'set03-V000-I01364.txt', 'set10-V000-I01674.txt', 'set04-V001-I02930.txt', 'set10-V000-I03489.txt', 'set03-V001-I00800.txt', 'set11-V001-I01330.txt', 'set11-V000-I00280.txt', 'set05-V000-I01425.txt', 'set11-V000-I00751.txt', 'set11-V001-I00114.txt', 'set03-V000-I01655.txt', 'set05-V000-I00593.txt', 'set03-V000-I01251.txt', 'set10-

In [27]:
import os

# specify the path to the folder
path = "/content/train/DistDataset/merged_files"

# initialize a variable to keep track of the total number of files
total_files = 0

# loop through all folders and subfolders in the path
for foldername, subfolders, filenames in os.walk(path):

    # count the number of files in each folder
    num_files = len(filenames)

    # add the number of files to the total
    total_files += num_files

# print the total number of files
print(f"Total number of files: {total_files}")


Total number of files: 15817


In [29]:
# kaist데이터셋 파일들을 기존 데이터셋 폴더로 복사해주기

import shutil
import os

source_dir = "/content/train/kaistDataset/kaistimages/resize_images"
target_dir = "/content/train/Dataset/images"

for filename in os.listdir(source_dir):
    # 이동할 파일 경로 지정
    file_path = os.path.join(source_dir, filename)
    # 파일이면 이동
    if os.path.isfile(file_path):
        shutil.copy(file_path, target_dir)

In [30]:
# 파일 개수 조회

import os

# Set folder path
source_folder = '/content/train/Dataset/images'

# Get list of files in folder
files = os.listdir(source_folder)

# Get number of files in folder
num_files = len(files)

print(f'The folder {source_folder} contains {num_files} files.')

The folder /content/train/Dataset/images contains 123224 files.


In [31]:
# kaist데이터셋 파일들을 기존 데이터셋 폴더로 복사해주기

import shutil
import os

source_dir = "/content/train/DistDataset/merged_files"
target_dir = "/content/train/Dataset/labels"

for filename in os.listdir(source_dir):
    # 이동할 파일 경로 지정
    file_path = os.path.join(source_dir, filename)
    # 파일이면 이동
    if os.path.isfile(file_path):
        shutil.copy(file_path, target_dir)

In [32]:
# 파일 개수 조회

import os

# Set folder path
source_folder = '/content/train/Dataset/labels'

# Get list of files in folder
files = os.listdir(source_folder)

# Get number of files in folder
num_files = len(files)

print(f'The folder {source_folder} contains {num_files} files.')

The folder /content/train/Dataset/labels contains 57315 files.


# 학습, 검증, 테스트 파일 분리

1. class id 별로 폴더에 담기
2. train, val, test 데이터셋 생성하기

  2-1. class id 별로 train, val, test 데이터 8:1:1 비율로 나누기

  2-2. 나눠진 train, val, test 셋들 하나로 합치기

  2-3. txt 파일 생성하기

## 1. class id 별로 txt 파일을 새로운 폴더에 담기

In [33]:
import os
import shutil

!mkdir /content/train/DistDataset/class_labels

txt_folder = '/content/train/Dataset/labels/'
class_folder = '/content/train/DistDataset/class_labels/'

for class_id_to_copy in range(7):
    num_files = 0
    os.makedirs(os.path.join(class_folder, str(class_id_to_copy)), exist_ok=True)
    for filename in os.listdir(txt_folder):
        if filename.endswith('.txt'):
            with open(os.path.join(txt_folder, filename), 'r') as f:
                lines = f.readlines()
                if not lines:  # Skip the file if the file is empty
                    continue
                class_id = int(lines[0].split()[0])
                if class_id != class_id_to_copy:
                    continue
            print(f'Copying file {filename} to class {class_id_to_copy} folder...')
            shutil.copy(os.path.join(txt_folder, filename), os.path.join(class_folder, str(class_id_to_copy), filename))
            num_files += 1
    print(f'Total {num_files} files were copied to class {class_id_to_copy} folder.')
    print("--------------------------------------------------")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Copying file set11-V000-I00190.txt to class 4 folder...
Copying file set11-V000-I01540.txt to class 4 folder...
Copying file set05-V000-I02448.txt to class 4 folder...
Copying file set10-V000-I03438.txt to class 4 folder...
Copying file set04-V000-I02793.txt to class 4 folder...
Copying file set04-V001-I01777.txt to class 4 folder...
Copying file set10-V001-I04154.txt to class 4 folder...
Copying file set03-V000-I01345.txt to class 4 folder...
Copying file set03-V001-I00440.txt to class 4 folder...
Copying file set10-V001-I01432.txt to class 4 folder...
Copying file set04-V000-I03680.txt to class 4 folder...
Copying file set03-V000-I00054.txt to class 4 folder...
Copying file set03-V001-I00616.txt to class 4 folder...
Copying file set05-V000-I02377.txt to class 4 folder...
Copying file set11-V001-I00536.txt to class 4 folder...
Copying file set11-V000-I01311.txt to class 4 folder...
Copying file set10-V000-I00099.txt to class 4 folder...
Copying file

In [34]:
import os

# specify the path to the folder
path = "/content/train/DistDataset/class_labels"

# loop through all folders and subfolders in the path
for foldername, subfolders, filenames in os.walk(path):

    # count the number of files in each folder
    num_files = len(filenames)

    # print the folder name and number of files
    print(f"{foldername}: {num_files}")

/content/train/DistDataset/class_labels: 0
/content/train/DistDataset/class_labels/4: 14284
/content/train/DistDataset/class_labels/6: 0
/content/train/DistDataset/class_labels/2: 6555
/content/train/DistDataset/class_labels/3: 4922
/content/train/DistDataset/class_labels/5: 1533
/content/train/DistDataset/class_labels/1: 16567
/content/train/DistDataset/class_labels/0: 13454


In [35]:
shutil.rmtree('/content/train/DistDataset/class_labels/6')

## 2. class id 별로 train, val, test 데이터 8:1:1 비율로 나누고, txt 리스트 적힌 파일 생성하기

In [36]:
import os
import random

class_folder = '/content/train/DistDataset/class_labels/'
output_folder = '/content/train/DistDataset/'

train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

train_set = set()
val_set = set()
test_set = set()

# class_labels 폴더 내부의 모든 폴더들(0~87)에 대해서 실행
for class_id in range(5):
    class_dir = os.path.join(class_folder, str(class_id))
    file_list = os.listdir(class_dir)
    random.shuffle(file_list)

    num_files = len(file_list)

    train_files = file_list[:int(num_files*train_ratio)]
    val_files = file_list[int(num_files*train_ratio):int(num_files*(train_ratio+val_ratio))]
    test_files = file_list[int(num_files*(train_ratio+val_ratio)):]

    # 각각 분할된 파일들의 파일 이름을 set에 추가
    for filename in train_files:
        full_path = os.path.join(class_dir, filename)
        if full_path not in val_set and full_path not in test_set:
            train_set.add(full_path)
    for filename in val_files:
        full_path = os.path.join(class_dir, filename)
        if full_path not in train_set and full_path not in test_set:
            val_set.add(full_path)
    for filename in test_files:
        full_path = os.path.join(class_dir, filename)
        if full_path not in train_set and full_path not in val_set:
            test_set.add(full_path)

# train, val, test 각각에 대해 해당하는 파일 이름 리스트를 파일에 저장
with open(os.path.join(output_folder, 'train.txt'), 'w') as train_file:
    for item in train_set:
        train_file.write("%s\n" % item)
with open(os.path.join(output_folder, 'valid.txt'), 'w') as val_file:
    for item in val_set:
        val_file.write("%s\n" % item)
with open(os.path.join(output_folder, 'test.txt'), 'w') as test_file:
    for item in test_set:
        test_file.write("%s\n" % item)


## .txt 파일 기준으로 작성된 데이터 세트들 jpg 형식으로 바꿔주기 

In [37]:
# txt 파일 jpg 내용으로, 파일 경로 부분 수정

with open('/content/train/DistDataset/test.txt', 'r') as f:
    lines = f.readlines()

new_lines = []
for line in lines:
    new_line = "/content/train/Dataset/images/" + line.replace('.txt', '.jpg').split('/')[-1].strip()
    new_lines.append(new_line)

with open('/content/train/DistDataset/test.txt', 'w') as f:
    f.writelines('\n'.join(new_lines))

# txt 파일 jpg 내용으로, 파일 경로 부분 수정

with open('/content/train/DistDataset/train.txt', 'r') as f:
    lines = f.readlines()

new_lines = []
for line in lines:
    new_line = "/content/train/Dataset/images/" + line.replace('.txt', '.jpg').split('/')[-1].strip()
    new_lines.append(new_line)

with open('/content/train/DistDataset/train.txt', 'w') as f:
    f.writelines('\n'.join(new_lines))

# txt 파일 jpg 내용으로, 파일 경로 부분 수정

with open('/content/train/DistDataset/valid.txt', 'r') as f:
    lines = f.readlines()

new_lines = []
for line in lines:
    new_line = "/content/train/Dataset/images/" + line.replace('.txt', '.jpg').split('/')[-1].strip()
    new_lines.append(new_line)

with open('/content/train/DistDataset/valid.txt', 'w') as f:
    f.writelines('\n'.join(new_lines))


In [38]:
# 파일 위치 옮기기
import shutil

src_file = "/content/train/DistDataset/test.txt"
dst_folder = "/content/train/Dataset"

shutil.copy(src_file, dst_folder)

src_file = "/content/train/DistDataset/train.txt"
dst_folder = "/content/train/Dataset"

shutil.copy(src_file, dst_folder)

src_file = "/content/train/DistDataset/valid.txt"
dst_folder = "/content/train/Dataset"

shutil.copy(src_file, dst_folder)

'/content/train/Dataset/valid.txt'

# yolov7 모듈 설치

In [39]:
# yolov7 clone 및 yolo 실행에 필요한 모듈 설치

main_path = main_path
os.chdir(main_path)

!git clone https://github.com/WongKinYiu/yolov7.git
os.chdir(main_path + 'yolov7')
!pip3 install -r requirements.txt

print('완료')

Cloning into 'yolov7'...
remote: Enumerating objects: 1139, done.
remote: Total 1139 (delta 0), reused 0 (delta 0), pack-reused 1139
Receiving objects: 100% (1139/1139), 70.31 MiB | 17.75 MiB/s, done.
Resolving deltas: 100% (514/514), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 70.6 MB/s eta 0:00:00
완료


# yolov7 학습 실행

In [ ]:
# yolo 학습 코드
import shutil

# tracking model 용 train
origin = google_drive_path + 'yolov7/train_copy.py'
copy = main_path + 'yolov7/train_copy.py'
shutil.copy(origin, copy)

os.chdir(main_path + 'yolov7')
# !sudo python3 train.py --img 416 --batch 64 --epochs 100 --data ../Dataset/data.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt
# !sudo python train.py --workers 8 --device 0 --batch-size 64 --data ../Dataset/data.yaml --img 416 416 --cfg cfg/training/yolov7.yaml --weights yolov7.pt --name yolov7 --hyp data/hyp.scratch.p5.yaml
# 정상 train : !sudo python train.py --workers 8 --batch-size 32 --data ../Dataset/data.yaml --img 416 416 --cfg cfg/training/yolov7.yaml --weights '' --name yolov7 --hyp data/hyp.scratch.p5.yaml
!sudo python train_copy.py --workers 8 --batch-size 32 --epochs 30 --data '/content/drive/MyDrive/[SeSAC] Final_PJT 공유폴더/03. 파이널_야생동물 객체 탐지, 추적 모델/데이터셋/야생5종+coco세팅파일/kaist2Wild4_night_data.yaml' --img 416 416 --cfg '/content/drive/MyDrive/[SeSAC] Final_PJT 공유폴더/03. 파이널_야생동물 객체 탐지, 추적 모델/데이터셋/야생5종+coco세팅파일/kaist2Wild4_night_cfg.yaml' --weights '/content/drive/MyDrive/[SeSAC] Final_PJT 공유폴더/03. 파이널_야생동물 객체 탐지, 추적 모델/데이터셋/야생5종+coco세팅파일/yolov7.pt' --name yolov7xcustom1 --hyp data/hyp.scratch.p5.yaml

# !sudo python train.py --workers 8 --device 0 --batch-size 32 --data data/custom.yaml --img 640 640 --cfg cfg/training/yolov7-custom.yaml --weights 'yolov7_training.pt' --name yolov7-custom --hyp data/hyp.scratch.custom.yaml
# !python train.py --workers 8 --device 0 --batch-size 16 --data data/custom_data.yaml --img 640 640 --cfg cfg/training/yolov7_custom_3.yaml --weights '' --name yolov7xcustom3 --hyp data/hyp.scratch.p5.yaml --epochs 200

2023-04-06 05:13:33.809590: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-06 05:13:33.866510: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-06 05:13:34.784316: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
YOLOR 🚀 v0.1-122-g3b41c2c torch 2.0.0+cu118 CPU

Namespace(weights='/content/drive/MyDrive/[SeSAC] Final_PJT 공유폴더/03. 파이널_야생동물 객체 탐지, 추적 모델/데이터셋/야생5종+coco세팅파일/yolov7.pt', cfg='/content/drive/MyDrive/[SeSAC

# 학습 결과 구글 드라이브로 옮기기

In [ ]:
# weight 파일 구글드라이브로 옮기기

import shutil

i = len(os.listdir(google_drive_path + 'yolov7/runs/train/'))
folder_name = google_drive_path + 'yolov7/runs/train/exp' + str(i + 2)
os.mkdir(folder_name)

origin = main_path + '/yolov7/runs/train/yolov7xcustom1/weights/best.pt'
copy = folder_name + '/best.pt'
shutil.copy(origin, copy)

# yolo 테스트 코드

In [ ]:
# yolo 테스트 코드
# 본 코드 오류시, 아래 셀의 설명을 읽어주세요.

import os
os.chdir(main_path + 'yolov7')

!sudo python3 test.py --data '/content/drive/MyDrive/[SeSAC] Final_PJT 공유폴더/03. 파이널_야생동물 객체 탐지, 추적 모델/데이터셋/야생5종+coco세팅파일/coco2Wild4_night_data.yaml' --batch 128 --weights '/content/train/yolov7/runs/train/yolov7xcustom1/weights/best.pt' --task test --img 416 --save-txt --save-conf